In [1]:
import QDMPy.data_loading as data_loading
import QDMPy.fit_interface as fit_interface
import QDMPy.fit_plots as fit_plots

import matplotlib
import matplotlib.pyplot as plt

In [2]:
# '%matplotlib widget' allows interactive plots in jupyterlab
# use ''%matplotlib inline' to allow export to pdf
%matplotlib widget

# Load the data in

In [3]:
options_dict = {
    "base_dir": "test_datasets",
    "filepath": "Spin relaxation (T1)_19",
    
    "additional_bins": 4, 

    "system_name": "Liams_Widefield",

    "ROI": "Full", 
    "ROI_centre": [400, 400],
    "ROI_rect_size": [1000,1000],

    "normalisation": "div",
    
    "single_pixel_check": [50,50],
    "AOI_1_start": [200, 300],
    "AOI_1_end": [240, 340], 
    "AOI_1_start": [200, 200],
    "AOI_2_end": [240, 240], 
    "AOI_1_start": [300, 200],
    "AOI_3_end": [320, 240], 
    
    "fit_backend": "scipy",
    "fit_pixels": True, 
    "force_fit": True, 
    "auto_match_prev_ROI_options": False,
    
    "fit_backend": "scipy",
    "fit_backend_comparison": ["scipy"],
    "scramble_pixels": True,
    
    "sub_threads": 2, 

    "fit_functions":  {"constant": 1, "stretched_exponential": 1},

    "constant_guess": 1,
    "constant_bounds": [0.9, 1.1],

    "power_exp_guess": 0.7,
    "power_exp_bounds": [0.5, 1.0],

    "charac_exp_t_guess": 1e-3,
    "charac_exp_t_bounds": [100e-6, 30e-3],

    "amp_exp_guess": 0.07,
    "amp_exp_bounds": [0.001, 0.2]
}

In [4]:
# QDMPy_path = data_loading.DIR_PATH
# options = data_loading.load_options(path=QDMPy_path / "options/test_options_t1.json", check_for_prev_result=True)
options = data_loading.load_options(options_dict=options_dict, check_for_prev_result=True)
fit_plots.set_mpl_rcparams(options)

In [5]:
raw_data, prelim_sweep_list = data_loading.load_raw_and_sweep(options)

In [6]:
PL_image, PL_image_ROI, sig, ref, sig_norm, single_pixel_pl, sweep_list = data_loading.reshape_dataset(
        options, raw_data, prelim_sweep_list
    )

In [7]:
fig1 = fit_plots.plot_ROI_PL_image(options, PL_image)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
AOIs = data_loading.define_AOIs(options)

In [9]:
fig2 = fit_plots.plot_AOI_PL_images(options, PL_image_ROI, AOIs)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
aoi_spectra_plot = fit_plots.plot_AOI_spectra(options, AOIs, sig, ref, sweep_list)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Fit AOI, ROI averages, single pixel

In [11]:
fit_model = fit_interface.define_fit_model(options)

/home/samsc/src/nv/QDMPy_proj/QDMPy_git/QDMPy/fit_scipy.py:26: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm  # auto detects jupyter


In [12]:
backend_ROI_results_lst = fit_interface.fit_ROI_avg(options, sig_norm, sweep_list, fit_model)

In [13]:
fig3 = fit_plots.plot_ROI_avg_fits(options, backend_ROI_results_lst)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
fit_result_collection_lst = fit_interface.fit_AOIs(options, sig_norm, single_pixel_pl, sweep_list, fit_model, AOIs, backend_ROI_results_lst)

In [15]:
AOI_spectra_fit_fig = fit_plots.plot_AOI_spectra_fit(options, sig, ref, sweep_list, AOIs, fit_result_collection_lst, fit_model)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Fit pixels

In [16]:
if (options["force_fit"] or not options["found_prev_result"]) and options["fit_pixels"]:
    pixel_fit_params = fit_interface.fit_pixels(options, sig_norm, sweep_list, fit_model, backend_ROI_results_lst[0])
elif options["found_prev_result"]:
    pixel_fit_params = fit_interface.load_prev_fit_results(options)
else:
    pixel_fit_params = None  # not fitting pixels, this stops plotting (e.g. via plot_param_images) from erroring

  0%|          | 0/16384 [00:00<?, ? PX/s]

# Plot (fit) parameters

In [17]:
fit_plots.plot_param_images(options, fit_model, pixel_fit_params, "constant")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
fit_plots.plot_param_images(options, fit_model, pixel_fit_params, "power_exp")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
fit_plots.plot_param_images(options, fit_model, pixel_fit_params, "charac_exp_t")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
fit_plots.plot_param_images(options, fit_model, pixel_fit_params, "amp_exp")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
# fit_plots.plot_param_images(options, fit_model, pixel_fit_params, "fwhm")

In [22]:
data_loading.save_options(options)